As electronic vehicles (EVs) become more popular, there is an increasing need for access to charging stations, also known as ports. To that end, many modern apartment buildings have begun retrofitting their parking garages to include shared charging stations. A charging station is shared if it is accessible by anyone in the building.

<img src="charging_station.jpg" alt="EV Charging" width="500" height="auto">

But with increasing demand comes competition for these ports — nothing is more frustrating than coming home to find no charging stations available! In this project, you will use a dataset to help apartment building managers better understand their tenants’ EV charging habits.

The data has been loaded into a PostgreSQL database with a table named `charging_sessions` with the following columns:

## charging_sessions

| Column | Definition | Data type |
|-|-|-|
|`garage_id`| Identifier for the garage/building|`VARCHAR`|
|`user_id` | Identifier for the individual user|`VARCHAR`|
|`user_type`|Indicating whether the station is `Shared` or `Private`| `VARCHAR` |
|`start_plugin`|The date and time the session started |`DATETIME`|
|`start_plugin_hour`|The hour (in military time) that the session started | `NUMERIC`|
|`end_plugout`|The date and time the session ended | `DATETIME` |
|`end_plugout_hour`|The hour (in military time) that the session ended | `NUMERIC`|
|`duration_hours`| The length of the session, in hours|`NUMERIC`|
|`el_kwh`| Amount of electricity used (in Kilowatt hours)|`NUMERIC`|
|`month_plugin`| The month that the session started |`VARCHAR`|
|`weekdays_plugin`| The day of the week that the session started|`VARCHAR`|

Let’s get started!

#### Sources
- **Data**: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0), via [Kaggle](https://www.kaggle.com/datasets/anshtanwar/residential-ev-chargingfrom-apartment-buildings),
- **Image**: Julian Herzog, [CC BY 4.0](https://creativecommons.org/licenses/by/4.0), via Wikimedia Commons

In [8]:
SELECT * FROM public.charging_sessions

,garage_id,user_id,user_type,start_plugin,start_plugin_hour,end_plugout,end_plugout_hour,el_kwh,duration_hours,month_plugin,weekdays_plugin
0,AdO3,AdO3-4,Private,2018-12-21 10:20:00+00:00,10,2018-12-21 10:23:00+00:00,10.0,0.30,0.050000,Dec,Friday
1,AdO3,AdO3-4,Private,2018-12-21 10:24:00+00:00,10,2018-12-21 10:32:00+00:00,10.0,0.87,0.136667,Dec,Friday
2,AdO3,AdO3-4,Private,2018-12-21 11:33:00+00:00,11,2018-12-21 19:46:00+00:00,19.0,29.87,8.216389,Dec,Friday
3,AdO3,AdO3-2,Private,2018-12-22 16:15:00+00:00,16,2018-12-23 16:40:00+00:00,16.0,15.56,24.419722,Dec,Saturday
4,AdO3,AdO3-2,Private,2018-12-24 22:03:00+00:00,22,2018-12-24 23:02:00+00:00,23.0,3.62,0.970556,Dec,Monday
...,...,...,...,...,...,...,...,...,...,...,...
6873,UT15,UT15-2,Private,2020-01-31 17:57:00+00:00,17,2020-01-31 22:29:00+00:00,22.0,13.43,4.530000,Jan,Friday
6874,AsO10,AsO10-1,Private,2020-01-31 18:32:00+00:00,18,2020-01-31 21:43:00+00:00,21.0,9.10,3.182222,Jan,Friday
6875,Bl2,Bl2-1,Private,2020-01-31 19:25:00+00:00,19,2020-01-31 20:37:00+00:00,20.0,4.28,1.203889,Jan,Friday
6876,UT7,UT7-1,Private,2020-01-31 19:27:00+00:00,19,2020-01-31 19:48:00+00:00,19.0,2.31,0.352500,Jan,Friday


In [9]:
SELECT * 
FROM charging_sessions
LIMIT 5;

,garage_id,user_id,user_type,start_plugin,start_plugin_hour,end_plugout,end_plugout_hour,el_kwh,duration_hours,month_plugin,weekdays_plugin
0,AdO3,AdO3-4,Private,2018-12-21 10:20:00+00:00,10,2018-12-21 10:23:00+00:00,10.0,0.30,0.050000,Dec,Friday
1,AdO3,AdO3-4,Private,2018-12-21 10:24:00+00:00,10,2018-12-21 10:32:00+00:00,10.0,0.87,0.136667,Dec,Friday
2,AdO3,AdO3-4,Private,2018-12-21 11:33:00+00:00,11,2018-12-21 19:46:00+00:00,19.0,29.87,8.216389,Dec,Friday
3,AdO3,AdO3-2,Private,2018-12-22 16:15:00+00:00,16,2018-12-23 16:40:00+00:00,16.0,15.56,24.419722,Dec,Saturday
4,AdO3,AdO3-2,Private,2018-12-24 22:03:00+00:00,22,2018-12-24 23:02:00+00:00,23.0,3.62,0.970556,Dec,Monday


In [10]:
-- unique_users_per_garage
SELECT
	garage_id,
	COUNT(DISTINCT user_id) as num_unique_users
FROM charging_sessions
WHERE user_type = 'Shared'
GROUP BY garage_id
ORDER BY num_unique_users DESC;



,garage_id,num_unique_users
0,Bl2,18
1,AsO2,17
2,UT9,16
3,AdO3,3
4,MS1,2
5,SR2,2
6,AdA1,1
7,Ris,1


In [11]:
-- most_popular_shared_start_times
SELECT
weekdays_plugin,
start_plugin_hour,
COUNT(*) as num_charging_sessions
FROM charging_sessions
WHERE user_type = 'Shared'
GROUP BY weekdays_plugin, start_plugin_hour
ORDER BY num_charging_sessions DESC
LIMIT 10;



,weekdays_plugin,start_plugin_hour,num_charging_sessions
0,Sunday,17,30
1,Friday,15,28
2,Thursday,19,26
3,Thursday,16,26
4,Wednesday,19,25
5,Sunday,18,25
6,Sunday,15,25
7,Monday,15,24
8,Friday,16,24
9,Tuesday,16,23


In [12]:
-- long_duration_shared_users

SELECT *
FROM (
    SELECT
        user_id,
        AVG(duration_hours) AS avg_charging_duration
    FROM charging_sessions
    WHERE user_type = 'Shared'
    GROUP BY user_id
    HAVING AVG(duration_hours) > 10
) AS long_duration_shared_users
ORDER BY avg_charging_duration DESC;



,user_id,avg_charging_duration
0,Share-9,16.845833
1,Share-17,12.894556
2,Share-25,12.214475
3,Share-18,12.088807
4,Share-8,11.550431
5,AdO3-1,10.369387
